<a href="https://colab.research.google.com/github/usman-babar/BigMart-Product-Outlet-Sales-Analysis-and-Prediction-Project/blob/main/Mini_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mini Project**

# **Problem Statement**

The problem is to build a predictive model to determine the sales of each product at different stores.

# **Hypothesis Generation**

Generate hypotheses about the factors that might influence sales based on prior knowledge and understanding of the data. For example, the location of the store, product attributes, promotions, and store size could be potential factors.

# **Loading Packages and Data**

In [3]:
import pandas as pd

# Load the data into a DataFrame
data_url = 'https://drive.google.com/file/d/1qHTYFy5lLTfzSH2x6j8OZYsKvWWrDqXp/view?usp=drive_link'
file_id = data_url.split('/')[-2]
download_url = f'https://drive.google.com/uc?id={file_id}'
df = pd.read_csv(download_url)


print(df)

     Item_Identifier  Item_Weight Item_Fat_Content  Item_Visibility  \
0              FDA15        9.300          Low Fat         0.016047   
1              DRC01        5.920          Regular         0.019278   
2              FDN15       17.500          Low Fat         0.016760   
3              FDX07       19.200          Regular         0.000000   
4              NCD19        8.930          Low Fat         0.000000   
...              ...          ...              ...              ...   
8518           FDF22        6.865          Low Fat         0.056783   
8519           FDS36        8.380          Regular         0.046982   
8520           NCJ29       10.600          Low Fat         0.035186   
8521           FDN46        7.210          Regular         0.145221   
8522           DRG01       14.800          Low Fat         0.044878   

                  Item_Type  Item_MRP Outlet_Identifier  \
0                     Dairy  249.8092            OUT049   
1               Soft Drinks  

# **Exploratory Data Analysis**